# Census LEHD WAC Supportive Stats

This notebook is meant to explore summary statistics that may be used to support the 2015 LQ, 2002–2015 LQ diff, and job density analyses.

Ideas could be:
- determine raw job count changes at the county level for each county
- look for noticeable changes by job sector (e.g. places where manufacturing jobs took a hit and were replaced by another job category)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sys, os

## Load Data

In [2]:
# specify paths for csv and shapefile data
dirname = os.path.dirname(os.path.realpath("__file__"))
wac2015_filepath = os.path.join(dirname, "../data/wac/ca_wac_S000_JT00_2015.csv.gz")
wac2002_filepath = os.path.join(dirname, "../data/wac/ca_wac_S000_JT00_2002.csv.gz")
cxwalk_filepath = os.path.join(dirname, "../data/wac/ca_xwalk.csv.gz")
tracts_shp_filepath = os.path.join(dirname, "../data/census_tracts/tracts_2010_4326.shp")

In [3]:
# load 2002 & 2015 census wac data, plus crosswalk file
wac2015 = pd.read_csv(wac2015_filepath, sep=",", delimiter=None, header="infer", names=None, index_col=None, usecols=None, compression="gzip")
wac2002 = pd.read_csv(wac2002_filepath, sep=",", delimiter=None, header="infer", names=None, index_col=None, usecols=None, compression="gzip")
cxwalk = pd.read_csv(cxwalk_filepath, sep=",", delimiter=None, header="infer", names=None, index_col=None, usecols=None, compression="gzip", encoding="ISO-8859-1", low_memory=False)


## Filter and Rollup 

In [4]:
# filter crosswalk table by 9 counties of SF Bay Area
cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]

In [5]:
# keep only the block fips, county fips, count name columns
cxwalk = cxwalk[['tabblk2010', 'cty', 'ctyname']]

In [6]:
# join 2015 and 2002 wac files to cxwalk using fields w_geocode and tabblk2010
wac2015 = wac2015.merge(cxwalk, how="inner", left_on="w_geocode", right_on="tabblk2010")
wac2002 = wac2002.merge(cxwalk, how="inner", left_on="w_geocode", right_on="tabblk2010")

In [7]:
# aggregate wac data to the county level
wac2015 = wac2015.groupby('cty', as_index=False).agg(np.sum)
wac2002 = wac2002.groupby('cty', as_index=False).agg(np.sum)
print(wac2015)

    cty           w_geocode     C000    CA01    CA02    CA03    CE01    CE02  \
0  6001  671379947659613568   751240  143890  433641  173709  144398  207492   
1  6013  392971825385879936   359762   71810  202060   85892   76675  101762   
2  6041  115929019670955728   112471   21223   62225   29023   23968   32405   
3  6055   69574259019475792    73604   15422   40407   17775   15321   25798   
4  6075  307093630483542400   700616  141710  414256  144650  113414  154762   
5  6081  296782474742689856   387932   73715  229957   84260   61282   96489   
6  6085  675673824262030080  1006868  183724  605665  217479  161625  225818   
7  6095  168350881040361632   144473   30546   80356   33571   33209   40485   
8  6097  236813394092384032   193045   40956  102836   49253   46359   66381   

     CE03  CNS01         ...          CFA03  CFA04  CFA05  CFS01  CFS02  \
0  399350    365         ...              0      0      0      0      0   
1  181325    696         ...              0      

In [8]:
# new data frame for storing county level stats
# create a map of county names to county fips codes
c = cxwalk[['cty', 'ctyname']].copy().drop_duplicates()
c['ctyname'] = c['ctyname'].str[:-10]
c = c.reset_index(drop=True)

print(c)

    cty         ctyname
0  6001        Alameda 
1  6013   Contra Costa 
2  6041          Marin 
3  6055           Napa 
4  6075  San Francisco 
5  6081      San Mateo 
6  6085    Santa Clara 
7  6095         Solano 
8  6097         Sonoma 


## Sum NAICS codes to create super categories

In [9]:
# NAICS codes for each super category
makers = ['CNS01', 'CNS02', 'CNS03', 'CNS04', 'CNS05', 'CNS06', 'CNS08']
services = ['CNS07', 'CNS14', 'CNS17', 'CNS18']
professions = ['CNS09', 'CNS10', 'CNS11', 'CNS12', 'CNS13']
support = ['CNS15', 'CNS16', 'CNS19', 'CNS20']

In [10]:
# create new aggregate columns for various job sectors
wac2015['makers'] = wac2015[makers].sum(axis=1)
wac2015['services'] = wac2015[services].sum(axis=1)
wac2015['professions'] = wac2015[professions].sum(axis=1)
wac2015['support'] = wac2015[support].sum(axis=1)
wac2015['total'] = wac2015['C000']

wac2002['makers'] = wac2002[makers].sum(axis=1)
wac2002['services'] = wac2002[services].sum(axis=1)
wac2002['professions'] = wac2002[professions].sum(axis=1)
wac2002['support'] = wac2002[support].sum(axis=1)
wac2002['total'] = wac2002['C000']

In [11]:
# make sure things add up
assert sum(wac2015['total'] -(wac2015['makers'] + wac2015['services'] + wac2015['professions'] + wac2015['support'])) == 0
assert sum(wac2002['total'] -(wac2002['makers'] + wac2002['services'] + wac2002['professions'] + wac2002['support'])) == 0

In [12]:
# keep only the columns we need from the wac dataframe
to_keep = ['cty', 'makers', 'services', 'professions', 'support', 'total']
wac2015 = wac2015[to_keep]
wac2002 = wac2002[to_keep]
print(wac2002)

    cty  makers  services  professions  support   total
0  6001  207584    161272       144053   177694  690603
1  6013   65021     91159        68769    99357  324306
2  6041   17445     33113        24764    30249  105571
3  6055   22122     16995         5547    14844   59508
4  6075   68532    157342       161633   130325  517832
5  6081   95521     91260        84280    69710  340771
6  6085  283270    202144       194061   183572  863047
7  6095   31748     35438        12627    38019  117832
8  6097   54733     47551        23754    52476  178514


## Apply county level stats

In [13]:
# add the 2015 columns
c = pd.merge(c, wac2015[['makers', 'services', 'professions', 'support', 'total', 'cty']], on='cty')

# rename 2015 columns to distinguish from 2002
c = c.rename(columns={'makers': 'makers_2015', 'services': 'services_2015', 'professions': 'professions_2015', 'support': 'support_2015', 'total': 'total_2015'})

# ditto for 2002
c = pd.merge(c, wac2002[['makers', 'services', 'professions', 'support', 'total', 'cty']], on='cty')
c = c.rename(columns={'makers': 'makers_2002', 'services': 'services_2002', 'professions': 'professions_2002', 'support': 'support_2002', 'total': 'total_2002'})

# calc 2015 – 2002 change
c['maker_change'] = wac2015['makers'] - wac2002['makers']
c['prof_change'] = wac2015['professions'] - wac2002['professions']
c['services_change'] = wac2015['services'] - wac2002['services']
c['support_change'] = wac2015['support'] - wac2002['support']
c['total_change'] = wac2015['total'] - wac2002['total']

# calc 2015 – 2002 percentage change
c['maker_pct_change'] = (c['maker_change'] / wac2002['makers']).round(2)
c['prof_pct_change'] = (c['prof_change'] / wac2002['professions']).round(2)
c['services_pct_change'] = (c['services_change'] / wac2002['services']).round(2)
c['support_pct_change'] = (c['support_change'] / wac2002['support']).round(2)

c = c.sort_values('total_change', ascending=False)

In [14]:
# save above table to csv
c.to_csv(os.path.join(dirname, "county_level_change.csv"), index=False)

## Regional change of each category from 2002 – 2015

In [15]:
# new data frame for 9 county sf bay area region
reg = pd.DataFrame(
    columns=['category', '2002', '2015', 'change', 'pct_change'],
    index=[0, 1, 2, 3, 4]
)

reg['category'] = ['makers', 'services', 'professions', 'support', 'total']

#print(reg)

In [16]:
# calc 2015 & 2002 yearly totals for each category
reg['2015'] = [
    wac2015['makers'].sum(),
    wac2015['services'].sum(),
    wac2015['professions'].sum(),
    wac2015['support'].sum(),
    wac2015['total'].sum()
]

reg['2002'] = [
    wac2002['makers'].sum(),
    wac2002['services'].sum(),
    wac2002['professions'].sum(),
    wac2002['support'].sum(),
    wac2002['total'].sum()
]

#print(reg)

In [17]:
# calc the net and percentage diff between years
reg['change'] = reg['2015'] - reg['2002']
reg['pct_change'] = (reg['change'] / reg['2002'] * 100).round(2)

print(reg)

      category     2002     2015  change  pct_change
0       makers   845976   799916  -46060       -5.44
1     services   836274   985798  149524       17.88
2  professions   719488   887679  168191       23.38
3      support   796246  1056618  260372       32.70
4        total  3197984  3730011  532027       16.64


In [18]:
# save above table to csv
reg.to_csv(os.path.join(dirname, "region_level_change.csv"), index=False)